In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from food.clipmodel import *

# !nbdev_build_lib

In [3]:
#https://data.world/kgarrett/whats-on-the-menu/workspace/query?queryid=5d1d1edb-5e6c-4bc7-bf9e-b128f846b959
# pd.read_csv('food.csv').to_sql('foods',engine,index=False,if_exists='append')

In [4]:
# from tendo import singleton
# me = singleton.SingleInstance()

import os
os.environ["HF_DATASETS_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from food.tools import *
from food.psql import *
from food.paths import *
from PIL import Image
from food.clipmodel import image2clip
import custom_pandas as cpd
from tqdm import tqdm

In [5]:
query = """select p.id,name
             from foods p
            where p.clip is Null """

In [6]:
total = engine.execute(f'select count(*) from ({query}) a').one()

In [7]:
bs = 1
pd_iter = cpd.read_sql_query(query, engine, chunksize=bs, index_col='id')

In [8]:
for inp in tqdm(pd_iter, desc="clip food inference", total=total[0] // bs):  
    try:
        clip = text2clip(inp['name'].tolist()[0]).numpy().tolist()
        inp['clip'] = [clip]
        insert_ignore(inp,'foods',update=True,update_cols=['clip'],unique_cols=['id'])
    except:
        print(inp)


clip food inference:  15%|█▍        | 63207/426740 [42:44<5:17:11, 19.10it/s] 

                   name                                               clip
id                                                                        
63205  Eggs Boiled, One  [0.04016100615262985, -0.0010926617542281747, ...


clip food inference:  15%|█▍        | 63457/426740 [42:59<5:54:36, 17.07it/s] 